In [1]:
from collections import OrderedDict
from typing import List

import cv2
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
from robotpy_apriltag import AprilTagDetector, AprilTagPoseEstimator
from scipy.spatial.transform import Rotation


In [2]:
import matplotlib

matplotlib.use('TkAgg')

In [3]:
# Hyperparameters
horizontal_focal_length_pixels = 1581.7867974691412
horizontal_focal_center_pixels = 678.6724626822399
vertical_focal_length_pixels = 1581.7867974691412
vertical_focal_center_pixels = 529.4318832108801
tag_detection_size_m = np.array([0.28, 0.36, 0.25, 0.31])
tag_total_size_m = np.array([0.5, 0.44, 0.56, 0.56])
panel_size_m = 1.0
fam = ["tagCircle21h7", "tag25h9", "tagCircle49h12", "tagStandard52h13"]

# ! This has to recalculated for each image, by using the camera properties and flightheight
#panel_size_pixel = calculate_panel_size_in_pixels()
panel_size_pixel = 180.0  #180 pixel = 100 cm for test purposes for single image 0 
conversion_factor = panel_size_pixel / panel_size_m

# Conversions from the detection area width to the total tag width
tag_detection_to_total_size = tag_total_size_m / tag_detection_size_m
tag_detection_to_total_width_conversions = dict(zip(fam, tag_detection_to_total_size))

# Sizes of the tags converted to meters 
tag_sizes_in_m = dict(zip(fam, zip(tag_detection_size_m, tag_total_size_m)))

## We propose two methods for using apriltags to mark the panel edges:
# ST: Single Tag, uses a single tag for each panel by placing it at the midpoint of an edge of the panel
# CT: Corner Tags, uses multiple tags to mark the panel corners

In [4]:
# There are either a tag at each corner of a panel
# or a single tag at the midpoint of an edge of each panel
paths_CT = [
    ".\\data\\apriltags_test\\best_case.png",
    ".\\data\\apriltags_test\\higher.png",
    ".\\data\\apriltags_test\\highest.tif",
]
paths_ST = [
    "data/apriltags_run2/0001SET/000/IMG_0031_2.tif",
    "data/apriltags_run2/0001SET/000/IMG_0057_5.tif"
]
images_CT = [cv2.imread(path, cv2.IMREAD_GRAYSCALE) for path in paths_CT]
images_ST = [cv2.imread(path, cv2.IMREAD_GRAYSCALE) for path in paths_ST]

In [5]:
families = [
    #"tag16h5",
    "tag25h9",
    #"tagStandard41h12",
    "tagStandard52h13",
    "tagCircle49h12",
    "tagCircle21h7",
]
detectors: List[AprilTagDetector] = []
for family in families:
    d = AprilTagDetector()
    d.addFamily(family)
    config = AprilTagDetector.Config()
    config.quadDecimate = 1.0
    config.numThreads = 4
    config.refineEdges = 1.0
    d.setConfig(config)
    detectors.append(d)

cmap = plt.get_cmap('viridis')
colors = cmap(np.linspace(0, 1, len(detectors)))

In [6]:
def verify_detections(tag, valid_ids=None) -> bool:
    if valid_ids is None:
        valid_ids = [0, 4, 9]
    return tag.getId() in valid_ids

In [7]:
def show_tags(img, show_id=False):
    plt.imshow(img, cmap="gray")
    plt.axis("off")
    for tags, color in zip(detect_tags(img), colors):
        for tag in tags:
            corners = list(tag.getCorners(tuple([0.0] * 8)))
            x, y = corners[::2], corners[1::2]
            # Append the first point to the end to close the rectangle/polygon
            x = list(x) + [x[0]]
            y = list(y) + [y[0]]
            plt.plot(x, y)
            plt.scatter(tag.getCenter().x, tag.getCenter().y, marker="o", color=color, s=20, label=tag.getFamily())
            if show_id:
                plt.text(tag.getCenter().x + 50, tag.getCenter().y - 50, tag.getId(), color="black", ha="center",
                         va="center",
                         bbox=dict(boxstyle="round",
                                   ec=(1., 0.5, 0.5),
                                   fc=(1., 0.8, 0.8),
                                   ))
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = OrderedDict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())
    plt.show()

In [8]:
def detect_tags(img, valid_ids=None):
    tags_by_detector = [detector.detect(img) for detector in detectors]
    return [[tag for tag in tags if verify_detections(tag, valid_ids)] for tags in tags_by_detector]

In [9]:
show_tags(images_ST[1], False)

In [10]:
def get_panels_ST(img, valid_ids=None):
    tags = detect_tags(img, valid_ids)
    tags = sum(tags, [])
    panels = []
    for tag in tags:
        corners = list(tag.getCorners(tuple([0.0] * 8)))
        corners = np.array(list((zip(corners[::2], corners[1::2]))))

        towards_panel = corners[2] - corners[1]
        tag_detection_size_pixel = np.linalg.norm(towards_panel)
        tag_size = tag_detection_size_pixel * tag_detection_to_total_width_conversions[tag.getFamily()]
        towards_panel = towards_panel / np.linalg.norm(towards_panel)
        center = np.array([tag.getCenter().x, tag.getCenter().y])
        tag_panel_border = center + towards_panel * (tag_size / 2)
        panel_length = towards_panel * panel_size_pixel
        half_panel_length = panel_length / 2
        panel_midpoint_to_edge = [-half_panel_length[1], half_panel_length[0]]
        edgeA = tag_panel_border + panel_midpoint_to_edge
        edgeB = tag_panel_border - panel_midpoint_to_edge
        edgeC = tag_panel_border + panel_length - panel_midpoint_to_edge
        edgeD = tag_panel_border + panel_length + panel_midpoint_to_edge
        panels.append((tag, (edgeA, edgeB, edgeC, edgeD)))
    return panels

In [11]:
def show_panels_ST(img, valid_ids):
    fig_2d = plt.figure()
    ax = fig_2d.subplots(1, 1)
    ax.imshow(img, cmap="grey")
    panels = get_panels_ST(img, valid_ids)
    for (tag, corners), color in zip(panels, colors):
        ax.scatter(tag.getCenter().x, tag.getCenter().y, color=color)
        x, y = zip(*corners)

        # Append the first point to the end to close the rectangle/polygon
        # Append the first point to the end to close the rectangle/polygon
        x = list(x) + [x[0]]
        y = list(y) + [y[0]]
        ax.plot(x, y, color=color, linewidth=2)
    fig_2d.show()

In [12]:
show_panels_ST(images_ST[1], [4])

## 3D Pose Estimation

In [13]:
# Helper functions

def rotate_vector(vector, axis, degrees):
    theta = degrees * (np.pi / 180.0)
    axis = axis / norm(axis)  # normalize the rotation vector first
    rot = Rotation.from_rotvec(theta * axis)
    new_v = rot.apply(vector)
    return new_v


def project_to_image_plane(X, Y, Z,
                           horizontal_focal_length_pixels, vertical_focal_length_pixels,
                           horizontal_focal_center_pixels, vertical_focal_center_pixels):
    """
    Projects a 3D world point (X, Y, Z) into 2D image plane coordinates (u, v).

    Parameters:
    X, Y, Z: float
        3D coordinates of the point in world space.
    horizontal_focal_length_pixels: float
        Focal length of the camera in the horizontal direction (in pixels).
    vertical_focal_length_pixels: float
        Focal length of the camera in the vertical direction (in pixels).
    horizontal_focal_center_pixels: float
        The x-coordinate of the principal point (optical center) in the image.
    vertical_focal_center_pixels: float
        The y-coordinate of the principal point (optical center) in the image.

    Returns:
    u, v: float
        2D pixel coordinates in the image plane.
    """
    # Project onto the 2D image plane
    u = (X * horizontal_focal_length_pixels) / Z + horizontal_focal_center_pixels
    v = (Y * vertical_focal_length_pixels) / Z + vertical_focal_center_pixels

    return u, v

In [20]:
def get_panels_ST_3D(img, valid_ids):
    panels = []
    tags = [detector.detect(img) for detector in detectors]
    #Flatten
    tags = sum(tags, [])

    def estimate(tag):
        width = tag_sizes_in_m[tag.getFamily()]
        pose_estimator = AprilTagPoseEstimator(
            AprilTagPoseEstimator.Config(width[0], horizontal_focal_length_pixels,
                                         vertical_focal_length_pixels,
                                         horizontal_focal_center_pixels, vertical_focal_center_pixels))
        return pose_estimator.estimate(tag)

    estimates = [estimate(tag) for tag in tags if verify_detections(tag, valid_ids)]
    for tag, estimate in zip(tags, estimates):
        tag_size = tag_sizes_in_m[tag.getFamily()]
        # The estimated rotation is a vector that points up and away form the panel (Z axis is up towards the camera)
        tag_orientation = [estimate.rotation().x, estimate.rotation().y, estimate.rotation().z]
        tag_center = [estimate.translation().x, estimate.translation().y, estimate.translation().z]
        # Normalize then and scale the rotation vector to the tag_size
        tag_orientation = tag_orientation / np.linalg.norm(tag_orientation)
        tag_orientation = tag_orientation * (tag_size[1] / 2)
        # The tag orientation is then rotated to face the "up" side of the tag when thinking in terms of a 2d paper print. (Towards the panel on the ground)
        towards_panel_direction = rotate_vector(tag_orientation, [0, 1, 0], 90.0)
        # Calculate location vectors that point in the direction of the panel edges
        panel_edge_midpoint = tag_center + towards_panel_direction
        panel_edge = (towards_panel_direction / np.linalg.norm(towards_panel_direction)) * panel_size_m
        half_panel_edge = panel_edge / 2
        panel_edge_midpoint_to_corner = rotate_vector(half_panel_edge, [0, 0, 1], 90.0)
        cornerA = panel_edge_midpoint + panel_edge_midpoint_to_corner
        cornerB = panel_edge_midpoint - panel_edge_midpoint_to_corner
        cornerC = panel_edge_midpoint + panel_edge - panel_edge_midpoint_to_corner
        cornerD = panel_edge_midpoint + panel_edge + panel_edge_midpoint_to_corner
        panels.append((estimate, (cornerA, cornerB, cornerC, cornerD)))
    return panels


def project(corners):
    return [project_to_image_plane(*corner,
                                   horizontal_focal_length_pixels,
                                   vertical_focal_length_pixels,
                                   horizontal_focal_center_pixels,
                                   vertical_focal_center_pixels) for corner in corners]

In [21]:
def show_panels_ST_3D_in_2D(img, valid_ids):
    fig_2d = plt.figure()
    ax = fig_2d.subplots(1, 1)
    ax.imshow(img, cmap="grey")
    panels_3d = get_panels_ST_3D(img, valid_ids)
    panels = [(project([estimate.translation()]), project(corners)) for (estimate, corners) in panels_3d]
    for ([estimate], corners), color in zip(panels, colors):
        ax.scatter(estimate[0], estimate[1], color=color)
        x, y = zip(*corners)

        # Append the first point to the end to close the rectangle/polygon
        x = list(x) + [x[0]]
        y = list(y) + [y[0]]
        ax.plot(x, y, color=color)
    fig_2d.show()

In [22]:
def show_panels_ST_3D_in_3D(img, valid_ids):
    fig_3d = plt.figure()
    ax_3d = fig_3d.add_subplot(111, projection='3d')
    ax_3d.set_xlim(-5, 5)
    ax_3d.set_ylim(-5, 5)
    ax_3d.set_zlim(25)
    color = "green"

    def print_location(point):
        ax_3d.scatter(*point, marker="o", color=color, s=2)

    panels_3d = get_panels_ST_3D(img, valid_ids)
    for (estimate, corners), color in zip(panels_3d, colors):
        ax_3d.quiver(*estimate.translation().toVector(),
                     *rotate_vector([estimate.rotation().x, estimate.rotation().y, estimate.rotation().z], [0, 1, 0],
                                    90.0),
                     color=color)
        print_location(estimate.translation())
        edgeA, edgeB, edgeC, edgeD = corners
        print_location(edgeA)
        print_location(edgeB)
        print_location(edgeC)
        print_location(edgeD)
        ax_3d.quiver(*edgeA, *(edgeB - edgeA), color=color)
        ax_3d.quiver(*edgeB, *(edgeC - edgeB), color=color)
        ax_3d.quiver(*edgeC, *(edgeD - edgeC), color=color)
        ax_3d.quiver(*edgeD, *(edgeA - edgeD), color=color)
    fig_3d.show()


In [23]:
show_panels_ST_3D_in_2D(images_ST[0], [4])
show_panels_ST_3D_in_3D(images_ST[0], [4])